In [1]:
import genjax

console = genjax.pretty()

In [2]:
genjax.adev.registry[type(genjax.Normal)]()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/mk/btkplz1n40q001dsy957srbh0000gn/T/ipykernel_85668/2080175230.py:1 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/mk/btkplz1n40q001dsy957srbh0000gn/T/ipykernel_85668/2080175230.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: ADEVPrimNormal.__init__() missing 1 required positional argument: 'inner'